# Precomputed_annotation_downsampling_debug

The purpose of this notebook is to figure out how to multiple spatial indexes for a precomputed annotation layer and to determine when they are needed for performance in terms of number of annotations. Refer to this document: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/datasource/precomputed/annotations.md for the general structure of these precomputed annotation layers and in particular the section: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/datasource/precomputed/annotations.md#spatial-index for spatial indexing.

So far, I have been able to successfully create a precomputed annotation layer with only a single spatial index, `spatial0`. This works well for up to a 10^5 annotations, but once you get above that the load time  starts to suffer. To get faster load times, spatial indexing is recommended, similar to downsampling for image and segmentation layers. To compute the spatial indexes, the instructions are:

- For each grid position cell at the coarsest level, compute the set remaining_annotations(0, cell) of annotations that intersect the cell. Note that a single annotation may intersect multiple cells.

- Sequentially generate spatial index level, starting at level=0 (the coarsest level):

- Define maxCount(level) to be the maximum over all cell positions of the size of remaining_annotations(level, cell).
- For each cell:
    - Compute a subset emitted(level, cell) of remaining_annotations(0, cell) where each annotation is chosen uniformly at random with probability min(1, limit / maxCount(level)).
    - This spatial index level maps cell to the list of annotations in emitted(level, cell). The annotations are encoded in the multiple annotation encoding also used by the related object id index; the list should be ordered randomly.
    - For each child_cell in level level+1 contained within cell: Compute the set remaining_annotations(level+1, child_cell) of annotations within remaining_annotations(level, cell) - emitted(level, cell) that intersect child_cell.
- Continue generating successively finer spatial index levels until no annotations remain.


To run this we will need to set the variable `limit`. It determines the subsampling level. Let's set it to 10000 for our example. We also need to know our total number of cells, which is 1388798 for an5 in Jess' 201904_ymaze_cfos cohort. We also need to decide the order in which to save the subset of cells in each grid cell file in each spatialX/ folder. The instruction say to use bit-reserved morton code, which I have no idea what that means. This section of the document says to order them randomly: https://github.com/google/neuroglancer/blob/master/src/neuroglancer/datasource/precomputed/annotations.md#multiple-annotation-encoding, so I'm going to try that. We also need to pre-decide how we are going to chunk up the volume. The instructions say to chunk it up in a way that results in the most isotropic chunk sizes. This is similar to how we would do things for downsampling. In our case here, we have an initial volume size of 2160, 2560, 687. So level1 will have grid_shape = [2,2,1] and a chunk size of [1080,1280,687]. level2 will have a grid shape of [4,4,1] and a chunk size of [540,640,687]. Then since this is close to isotropic the remaining grid shapes will increase by factors of 2 in each dimension , so [8,8,2], ... and chunk sizes will just be divided by 2 in each dimension, so [270,320,343.5], .... This is repeated until there no annotations remain - what does that mean?

Since our coarsest level has grid_shape = [1,1,1] then the set of `remaining_annotations(0, cell)` that intersect the cell is all of the annotations since we only have 1 cell. 

**level 0**
maxCount(level=0) = total cell count because we only have one 1 cell and it has all of the cells in it

There is only 1 cell, so we just have to:
- calculate the subset emitted(level=0,cell=0) of remaining_annotations(0, cell=0) where each annotation is chosen uniformly with probability min(1,limit=1000 / maxCount(level=0)) = 10000/1388798. Therefore, only ~0.7% of annotations will end up in this level.
- Save this subset to a file spatiall0/0_0_0 using the multiple annotation encoding that we normally use and in a random order. 
- For each child_cell (the four cells at level 1) contained within this cell, compute the set remaining_annotations(level=1,child_cell) of annotations within remaining_annotations(level=0,cell=0) - emitted(level=0,cell=0) that intersect child_cell. Since there are 4 child cells: [0,0,0],[1,0,0],[0,1,0],[1,1,0] and the current level is level0, remaining_annotations(level=0,cell=0) is all cells and emitted(level=0,cell=0) is the 0.7% that were saved out to spatial0/0_0_0. So of the 99.3% of the cells, figure out which of those intersect with each child cell. 

**level 1**
4 cells: [0,0,0],[1,0,0],[0,1,0],[1,1,0]
maxCount(level=1)) = max of remaining_annotations(level=1,cell_i) where at the end of the previous step we calculated all four of these remaining_annotations.

For each cell i of the 4 cells:
- calculate the subset emitted(level=1,cell=i) of remaining_annotations(level=1, cell=i) where each annotation is chosen uniformly with probability min(1,limit=1000 / maxCount(level=1)) = 10000/maxCount(level=1). 
- Save this subset to a file spatiall1/cell_i using the multiple annotation encoding that we normally use and in a random order.
- For each child_cell of this cell, compute the set remaining_annotations(level=2,child_cell) of annotations within remaining_annotations(level=1,cell=i) - emitted(level=1,cell=i) that intersect child_cell. Since there are 4 child cells: [0,0,0],[1,0,0],[0,1,0],[1,1,0] and the current level is level1, remaining_annotations(level=1,cell=i) was calculated in the previous step and emitted(level=1,cell=i) is the value we calculated just above that were saved out to spatial1/cell_i. So of the fraction of these of these cells, figure out which of those intersect with each child cell.

Do this until maxCount(level) is 0

In [17]:
import numpy as np
import os
import csv
import struct
import json
from cloudvolume import CloudVolume
import matplotlib.pyplot as plt
import neuroglancer
%matplotlib inline

In [18]:
# get the raw-space cells file and load it in
animal_id = 5
pth=os.path.join('/jukebox/wang/Jess/lightsheet_output',
        '201904_ymaze_cfos','processed',f'an{animal_id}','clearmap_cluster_output',
        'cells.npy')
converted_points = np.load(pth)

In [19]:
len(converted_points)

1388798

In [58]:
converted_points.shape

(1388798, 3)

In [197]:
unique_coordinates = np.unique(converted_points,axis=0)

In [200]:
len(unique_coordinates)

1388798

In [199]:
np.random.shuffle(unique_coordinates) # does it in place

In [201]:
def calculate_factors(level):
    # Calculate the factor to apply to the grid_shape/chunk size at a given spatial index level
    d = {}
    d[0] = [1,1,1]
    d[1] = [2,2,1]
    d[2] = [2,2,1]
    for i in range(3,20):
        d[i] = [2,2,2]
    return d[level]

In [202]:
# We need a function to calculate the subset emitted(level,cell,limit) 
def subset(level,cell,limit=10000):
    # Assumes two global dictionaries exist: remaining_annotations, maxCount
    rem = remaining_annotations[level][cell] # annotations 
    if maxCount[level] > limit:
        prob = limit/maxCount[level]
    else:
        N=maxCount[level]
    subset = np.random.choice(rem,size=N,replace=False) # uses a uniform distribution with no replacement
    return subset
    


In [203]:
def make_cells(grid_shape):
    # Make a list of cells ["0_0_0","1_0_0", ...] given a grid shape
    cells = []
    for x in range(grid_shape[0]):
        for y in range(grid_shape[1]):
            for z in range(grid_shape[2]):
                cell = f"{x}_{y}_{z}"
                cells.append(cell)
    return cells

In [204]:
def get_child_cells(cell,factor):
    # Given a cell string e.g. 1_2_3 and a factor, e.g. [2,2,1]
    # figure out all of the child cells 
    child_cells = []
    xcell,ycell,zcell = [int(x) for x in cell.split('_')] # n,m,p
    xfactor,yfactor,zfactor = factor # x,y,z
    for xf in range(0,xfactor):
        x_child = xcell*xfactor + xf
        for yf in range(0,yfactor):
            y_child = ycell*yfactor + yf
            for zf in range(0,zfactor):
                z_child = zcell*zfactor + zf
                child_cell = f"{x_child}_{y_child}_{z_child}"
                child_cells.append(child_cell)
    return child_cells

In [205]:
def save_cellfile(level,cell,coordinates):
    # We already know how to encode just the coordinates. Do it like so for the first 100 points
    spatial_dir = os.path.join(layer_dir,f"spatial{level}")
    if not os.path.exists(spatial_dir):
        os.mkdir(spatial_dir)
    filename = os.path.join(spatial_dir,cell)
    total_count = len(coordinates)
    with open(filename,'wb') as outfile:
        buf = struct.pack('<Q',total_count)
        pt_buf = b''.join(struct.pack('<3f',x,y,z) for (x,y,z) in coordinates)
        buf += pt_buf
        id_buf = struct.pack('<%sQ' % len(coordinates), *range(len(coordinates)))
        buf += id_buf
        outfile.write(buf)
    print(f"wrote {filename}")

In [280]:
def find_intersecting_coordinates(coord_array,lower_bounds,upper_bounds):
    # Of the coordinates in coord_list find the ones that are between lower_bounds and upper_bounds in x,y,z
    # return the indices of coord_array inside the bounds
    mask = (coord_array[:,0]>lower_bounds[0]) & (coord_array[:,0]<=upper_bounds[0]) & \
           (coord_array[:,1]>lower_bounds[1]) & (coord_array[:,1]<=upper_bounds[1]) & \
           (coord_array[:,2]>lower_bounds[2]) & (coord_array[:,2]<=upper_bounds[2])
    return coord_array[mask]


In [307]:
# initialize
layer_dir = "/home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex"
limit=10000

level=0
cell="0_0_0"
grid_shape = [1,1,1]
chunk_size = [2160,2560,687]
total_annotations = len(unique_coordinates)
remaining_annotations = {} # holds the array of coordinates in each cell at each level
remaining_annotations[level] = {cell:unique_coordinates}

maxCount = {}
while True:
# for level in range(0,3): # turn this into a while loop
    print("##############")
    print(f"Level: {level}")
    print("##############")
#     print("Remaining annotations at this level:")
#     print(remaining_annotations[level].values())
    N_annotations_this_level = [len(x) for x in remaining_annotations[level].values()]
    maxCount[level] = max(N_annotations_this_level)
    if maxCount[level] == 0:
        print("Finished!")
        break
    factor = calculate_factors(level)
    grid_shape = [a*b for a,b in zip(grid_shape,factor)]
    chunk_size = [a/b for a,b in zip(chunk_size,factor)]
    print(f"chunk_size={chunk_size}, maxCount = {maxCount[level]}")
    cells = make_cells(grid_shape)
    print("Have these cells:", cells)
    
#     print(cells)
    if maxCount[level] > limit:
        prob = limit/maxCount[level]
    else:
        prob = 1
    for cell in cells:
        print("In cell: ", cell)
        # Look up the remaining annotations in this cell, which was computed during the last iteration
        annotations_this_cell = remaining_annotations[level][cell]            
        N_annotations_this_cell = len(annotations_this_cell)
        print(f"started with {N_annotations_this_cell} annotations")
        next_factor = calculate_factors(level+1)
        child_cells = get_child_cells(cell,next_factor)
        next_chunk_size = [a/b for a,b in zip(chunk_size,next_factor)]

        if N_annotations_this_cell != 0:
            
            # Figure out the subset based on the probability we just calculated
            N_subset = int(round(N_annotations_this_cell*prob))
            # figure out list of indices of the remaining array to grab 
            subset_indices = np.random.choice(range(N_annotations_this_cell),size=N_subset,replace=False)
            print("annotations this cell:",annotations_this_cell[0:10])
            print("subset indices:",subset_indices[0:10])
            subset_cells = np.take(annotations_this_cell,subset_indices,axis=0)
            print(f"subsetted {len(subset_cells)} annotations")

            # save these cells to a file using the multi annotation convention we use for a single spatial index
            save_cellfile(level,cell,subset_cells)
            # For each child_cell contained within cell:
            # Compute the set remaining_annotations(level+1, child_cell)
            # of annotations within remaining_annotations(level, cell) - emitted(level, cell)
            # that intersect child_cell.

            # So we need to figure out which child cells are contained in this cell
            
            indices_annotations_this_cell = range(len(annotations_this_cell))
            leftover_annotation_indices = set(indices_annotations_this_cell)-set(subset_indices)
            leftover_annotations = np.take(annotations_this_cell,list(leftover_annotation_indices),axis=0)


            print(f"have {len(leftover_annotations)} annotations leftover")
        
        if level+1 not in remaining_annotations.keys():
            remaining_annotations[level+1] = {}
        print("Looping over child cells: ", child_cells)
        n_annotations_in_child_cells = 0
        for child_cell in child_cells:
            if N_annotations_this_cell == 0:
                remaining_annotations[level+1][child_cell] = np.array([])
                continue
            print(child_cell)
            # figure out which of the leftover annotations fall within this child cell
            # To do that I need to know the bounds of this child chunk
            # To know that I need to know the chunk size and the child chunk
            child_cell_indices = [int(x) for x in child_cell.split('_')]
            child_lower_bounds = [a*b for a,b in zip(child_cell_indices,next_chunk_size)]
            child_upper_bounds = [a+b for a,b, in zip(child_lower_bounds,next_chunk_size)]
#             print(f"Child cell: {child_cell}")
            print("Child lower and upper bounds")
            print(child_lower_bounds)
            print(child_upper_bounds)
         
            # Now use the bounds to find which annotations fall within these bounds
            intersecting_annotations_this_child = find_intersecting_coordinates(
                leftover_annotations,child_lower_bounds,child_upper_bounds)
            print(f"Have {len(intersecting_annotations_this_child)} in this child cell")
            remaining_annotations[level+1][child_cell] = intersecting_annotations_this_child
#             print()
            n_annotations_in_child_cells+=len(intersecting_annotations_this_child)
        if N_annotations_this_cell == 0:
            continue
        print(len(leftover_annotations),n_annotations_in_child_cells)
        print()
        #         assert len(leftover_annotations) == n_annotations_in_child_cells
    level+=1
        # of annotations within remaining_annotations(level=0,cell=0) - emitted(level=0,cell=0)
        # that intersect child_cell. Since there are 4 child cells:
        # and the current level is level0, 
        # remaining_annotations(level=0,cell=0) is all cells and emitted(level=0,cell=0) is the 0.7% that were saved out to spatial0/0_0_0. So of the 99.3% of the cells, figure out which of those intersect with each child cell.

##############
Level: 0
##############
chunk_size=[2160.0, 2560.0, 687.0], maxCount = 1388798
Have these cells: ['0_0_0']
In cell:  0_0_0
started with 1388798 annotations
annotations this cell: [[ 799 1569  387]
 [1150  899  531]
 [1457 1374  424]
 [1081  954  129]
 [1467 1569   73]
 [ 591 1803  309]
 [ 841  974  438]
 [ 863 2415  536]
 [ 826 1183  134]
 [ 493 1267  281]]
subset indices: [ 676004  190017  386842  182828  703775 1364584 1270336 1361066   24294
  632386]
subsetted 10000 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial0/0_0_0
have 1378798 annotations leftover
Looping over child cells:  ['0_0_0', '0_1_0', '1_0_0', '1_1_0']
0_0_0
Child lower and upper bounds
[0.0, 0.0, 0.0]
[1080.0, 1280.0, 687.0]
Have 442341 in this child cell
0_1_0
Child lower and upper bounds
[0.0, 1280.0, 0.0]
[1080.0, 2560.0, 687.0]
Have 390594 in this child cell
1_0_0
Child lower and upper bounds
[1080.0, 0.0, 0.0]
[2160.0, 1280.0, 687.0]
Have 277698 in 

annotations this cell: [[ 841  974  438]
 [ 826 1183  134]
 [ 925  990  353]
 [ 833  984  556]
 [ 694 1220  128]
 [ 678 1068  587]
 [ 713  879  373]
 [ 864 1211   36]
 [1040  906  526]
 [ 890 1248  229]]
subset indices: [ 30474  90503 116758 104298  45891 220685 235754  32401 115665 131907]
subsetted 10000 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial2/1_1_0
have 275265 annotations leftover
Looping over child cells:  ['2_2_0', '2_2_1', '2_3_0', '2_3_1', '3_2_0', '3_2_1', '3_3_0', '3_3_1']
2_2_0
Child lower and upper bounds
[540.0, 640.0, 0.0]
[810.0, 960.0, 343.5]
Have 27920 in this child cell
2_2_1
Child lower and upper bounds
[540.0, 640.0, 343.5]
[810.0, 960.0, 687.0]
Have 23635 in this child cell
2_3_0
Child lower and upper bounds
[540.0, 960.0, 0.0]
[810.0, 1280.0, 343.5]
Have 49592 in this child cell
2_3_1
Child lower and upper bounds
[540.0, 960.0, 343.5]
[810.0, 1280.0, 687.0]
Have 39542 in this child cell
3_2_0
Child lower and

annotations this cell: [[384 878 271]
 [503 897 266]
 [441 721 274]
 [484 830 331]
 [481 821 162]
 [410 731 286]
 [395 950 194]
 [465 867 225]
 [506 824 199]
 [368 866 329]]
subset indices: [ 8238 16024  8032 16843 16028 10215 11006 14754  8563  9908]
subsetted 3671 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial3/1_2_0
have 14535 annotations leftover
Looping over child cells:  ['2_4_0', '2_4_1', '2_5_0', '2_5_1', '3_4_0', '3_4_1', '3_5_0', '3_5_1']
2_4_0
Child lower and upper bounds
[270.0, 640.0, 0.0]
[405.0, 800.0, 171.75]
Have 0 in this child cell
2_4_1
Child lower and upper bounds
[270.0, 640.0, 171.75]
[405.0, 800.0, 343.5]
Have 550 in this child cell
2_5_0
Child lower and upper bounds
[270.0, 800.0, 0.0]
[405.0, 960.0, 171.75]
Have 21 in this child cell
2_5_1
Child lower and upper bounds
[270.0, 800.0, 171.75]
[405.0, 960.0, 343.5]
Have 3754 in this child cell
3_4_0
Child lower and upper bounds
[405.0, 640.0, 0.0]
[540.0, 800.0, 1

Child lower and upper bounds
[270.0, 2080.0, 343.5]
[405.0, 2240.0, 515.25]
Have 0 in this child cell
2_13_3
Child lower and upper bounds
[270.0, 2080.0, 515.25]
[405.0, 2240.0, 687.0]
Have 0 in this child cell
3_12_2
Child lower and upper bounds
[405.0, 1920.0, 343.5]
[540.0, 2080.0, 515.25]
Have 1740 in this child cell
3_12_3
Child lower and upper bounds
[405.0, 1920.0, 515.25]
[540.0, 2080.0, 687.0]
Have 32 in this child cell
3_13_2
Child lower and upper bounds
[405.0, 2080.0, 343.5]
[540.0, 2240.0, 515.25]
Have 274 in this child cell
3_13_3
Child lower and upper bounds
[405.0, 2080.0, 515.25]
[540.0, 2240.0, 687.0]
Have 0 in this child cell
2316 2316

In cell:  1_7_0
started with 0 annotations
Looping over child cells:  ['2_14_0', '2_14_1', '2_15_0', '2_15_1', '3_14_0', '3_14_1', '3_15_0', '3_15_1']
In cell:  1_7_1
started with 4 annotations
annotations this cell: [[ 517 2522  482]
 [ 539 2518  489]
 [ 530 2521  485]
 [ 535 2519  490]]
subset indices: [0]
subsetted 1 annotations
wr

have 39592 annotations leftover
Looping over child cells:  ['4_6_0', '4_6_1', '4_7_0', '4_7_1', '5_6_0', '5_6_1', '5_7_0', '5_7_1']
4_6_0
Child lower and upper bounds
[540.0, 960.0, 0.0]
[675.0, 1120.0, 171.75]
Have 4819 in this child cell
4_6_1
Child lower and upper bounds
[540.0, 960.0, 171.75]
[675.0, 1120.0, 343.5]
Have 4926 in this child cell
4_7_0
Child lower and upper bounds
[540.0, 1120.0, 0.0]
[675.0, 1280.0, 171.75]
Have 5069 in this child cell
4_7_1
Child lower and upper bounds
[540.0, 1120.0, 171.75]
[675.0, 1280.0, 343.5]
Have 5014 in this child cell
5_6_0
Child lower and upper bounds
[675.0, 960.0, 0.0]
[810.0, 1120.0, 171.75]
Have 4981 in this child cell
5_6_1
Child lower and upper bounds
[675.0, 960.0, 171.75]
[810.0, 1120.0, 343.5]
Have 5244 in this child cell
5_7_0
Child lower and upper bounds
[675.0, 1120.0, 0.0]
[810.0, 1280.0, 171.75]
Have 5403 in this child cell
5_7_1
Child lower and upper bounds
[675.0, 1120.0, 171.75]
[810.0, 1280.0, 343.5]
Have 4136 in this chi

...

23_18_5
Child lower and upper bounds
[1552.5, 1440.0, 429.375]
[1620.0, 1520.0, 515.25]
Have 0 in this child cell
23_19_4
Child lower and upper bounds
[1552.5, 1520.0, 343.5]
[1620.0, 1600.0, 429.375]
Have 0 in this child cell
23_19_5
Child lower and upper bounds
[1552.5, 1520.0, 429.375]
[1620.0, 1600.0, 515.25]
Have 0 in this child cell
0 0

In cell:  11_9_3
started with 898 annotations
annotations this cell: [[1500 1544  551]
 [1603 1446  546]
 [1505 1536  523]
 [1503 1543  552]
 [1561 1550  527]
 [1532 1496  554]
 [1509 1573  548]
 [1584 1519  561]
 [1594 1464  597]
 [1598 1585  537]]
subset indices: [ 72 568 241 257 735 559 608 224 886 390]
subsetted 898 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial4/11_9_3
have 0 annotations leftover
Looping over child cells:  ['22_18_6', '22_18_7', '22_19_6', '22_19_7', '23_18_6', '23_18_7', '23_19_6', '23_19_7']
22_18_6
Child lower and upper bounds
[1485.0, 1440.0, 515.25]
[1552.5, 1520.0, 601.

have 0 annotations leftover
Looping over child cells:  ['24_10_4', '24_10_5', '24_11_4', '24_11_5', '25_10_4', '25_10_5', '25_11_4', '25_11_5']
24_10_4
Child lower and upper bounds
[1620.0, 800.0, 343.5]
[1687.5, 880.0, 429.375]
Have 0 in this child cell
24_10_5
Child lower and upper bounds
[1620.0, 800.0, 429.375]
[1687.5, 880.0, 515.25]
Have 0 in this child cell
24_11_4
Child lower and upper bounds
[1620.0, 880.0, 343.5]
[1687.5, 960.0, 429.375]
Have 0 in this child cell
24_11_5
Child lower and upper bounds
[1620.0, 880.0, 429.375]
[1687.5, 960.0, 515.25]
Have 0 in this child cell
25_10_4
Child lower and upper bounds
[1687.5, 800.0, 343.5]
[1755.0, 880.0, 429.375]
Have 0 in this child cell
25_10_5
Child lower and upper bounds
[1687.5, 800.0, 429.375]
[1755.0, 880.0, 515.25]
Have 0 in this child cell
25_11_4
Child lower and upper bounds
[1687.5, 880.0, 343.5]
[1755.0, 960.0, 429.375]
Have 0 in this child cell
25_11_5
Child lower and upper bounds
[1687.5, 880.0, 429.375]
[1755.0, 960.0

[1755.0, 1840.0, 257.625]
Have 0 in this child cell
25_22_3
Child lower and upper bounds
[1687.5, 1760.0, 257.625]
[1755.0, 1840.0, 343.5]
Have 0 in this child cell
25_23_2
Child lower and upper bounds
[1687.5, 1840.0, 171.75]
[1755.0, 1920.0, 257.625]
Have 0 in this child cell
25_23_3
Child lower and upper bounds
[1687.5, 1840.0, 257.625]
[1755.0, 1920.0, 343.5]
Have 0 in this child cell
0 0

In cell:  12_11_2
started with 1981 annotations
annotations this cell: [[1673 1767  451]
 [1728 1836  443]
 [1643 1814  414]
 [1646 1910  468]
 [1713 1859  384]
 [1679 1916  490]
 [1657 1769  417]
 [1737 1823  360]
 [1651 1835  345]
 [1704 1849  373]]
subset indices: [ 266 1904 1792  666  722  982 1174 1729 1816 1659]
subsetted 1981 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial4/12_11_2
have 0 annotations leftover
Looping over child cells:  ['24_22_4', '24_22_5', '24_23_4', '24_23_5', '25_22_4', '25_22_5', '25_23_4', '25_23_5']
24_22_4
Child lowe

wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial4/13_8_1
have 0 annotations leftover
Looping over child cells:  ['26_16_2', '26_16_3', '26_17_2', '26_17_3', '27_16_2', '27_16_3', '27_17_2', '27_17_3']
26_16_2
Child lower and upper bounds
[1755.0, 1280.0, 171.75]
[1822.5, 1360.0, 257.625]
Have 0 in this child cell
26_16_3
Child lower and upper bounds
[1755.0, 1280.0, 257.625]
[1822.5, 1360.0, 343.5]
Have 0 in this child cell
26_17_2
Child lower and upper bounds
[1755.0, 1360.0, 171.75]
[1822.5, 1440.0, 257.625]
Have 0 in this child cell
26_17_3
Child lower and upper bounds
[1755.0, 1360.0, 257.625]
[1822.5, 1440.0, 343.5]
Have 0 in this child cell
27_16_2
Child lower and upper bounds
[1822.5, 1280.0, 171.75]
[1890.0, 1360.0, 257.625]
Have 0 in this child cell
27_16_3
Child lower and upper bounds
[1822.5, 1280.0, 257.625]
[1890.0, 1360.0, 343.5]
Have 0 in this child cell
27_17_2
Child lower and upper bounds
[1822.5, 1360.0, 171.75]
[1890.0, 1440.0, 257

 [1892 1118  385]]
subset indices: [62 49  3 21 36 54 51 40 25 64]
subsetted 66 annotations
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/spatial4/14_6_2
have 0 annotations leftover
Looping over child cells:  ['28_12_4', '28_12_5', '28_13_4', '28_13_5', '29_12_4', '29_12_5', '29_13_4', '29_13_5']
28_12_4
Child lower and upper bounds
[1890.0, 960.0, 343.5]
[1957.5, 1040.0, 429.375]
Have 0 in this child cell
28_12_5
Child lower and upper bounds
[1890.0, 960.0, 429.375]
[1957.5, 1040.0, 515.25]
Have 0 in this child cell
28_13_4
Child lower and upper bounds
[1890.0, 1040.0, 343.5]
[1957.5, 1120.0, 429.375]
Have 0 in this child cell
28_13_5
Child lower and upper bounds
[1890.0, 1040.0, 429.375]
[1957.5, 1120.0, 515.25]
Have 0 in this child cell
29_12_4
Child lower and upper bounds
[1957.5, 960.0, 343.5]
[2025.0, 1040.0, 429.375]
Have 0 in this child cell
29_12_5
Child lower and upper bounds
[1957.5, 960.0, 429.375]
[2025.0, 1040.0, 515.25]
Have 0 in this child

In [301]:
# Use the remaining_annotations dict to make the spatial part of the info file
limit=10000
grid_shape = [1,1,1]
chunk_size = [2160,2560,687]

spatial_array = []
for level in remaining_annotations.keys():
    N_annotations_this_level = [len(x) for x in remaining_annotations[level].values()]
    maxCount[level] = max(N_annotations_this_level)
    if maxCount[level] == 0:
        break
    factor = calculate_factors(level)
    grid_shape = [a*b for a,b in zip(grid_shape,factor)]
    chunk_size = [a/b for a,b in zip(chunk_size,factor)]
    spatial_dict = {"chunk_size":chunk_size,
                    "grid_shape":grid_shape,
                    "key":f"spatial{level}",
                    "limit":limit}
    spatial_array.append(spatial_dict)
spatial_array

[{'chunk_size': [2160.0, 2560.0, 687.0],
  'grid_shape': [1, 1, 1],
  'key': 'spatial0',
  'limit': 10000},
 {'chunk_size': [1080.0, 1280.0, 687.0],
  'grid_shape': [2, 2, 1],
  'key': 'spatial1',
  'limit': 10000},
 {'chunk_size': [540.0, 640.0, 687.0],
  'grid_shape': [4, 4, 1],
  'key': 'spatial2',
  'limit': 10000},
 {'chunk_size': [270.0, 320.0, 343.5],
  'grid_shape': [8, 8, 2],
  'key': 'spatial3',
  'limit': 10000},
 {'chunk_size': [135.0, 160.0, 171.75],
  'grid_shape': [16, 16, 4],
  'key': 'spatial4',
  'limit': 10000}]

In [305]:
info={
  "@type": "neuroglancer_annotations_v1",
  "annotation_type": "POINT",
  "by_id": {
    "key": "by_id"
  },
  "dimensions": {
    "x": [
      "5e-06",
      "m"
    ],
    "y": [
      "5e-06",
      "m"
    ],
    "z": [
      "1e-05",
      "m"
    ]
  },
  "lower_bound": [
    0,
    0,
    0
  ],
  "properties": [
  ],
  "relationships": [],
  "spatial": [{'chunk_size': [2160.0, 2560.0, 687.0],
  'grid_shape': [1, 1, 1],
  'key': 'spatial0',
  'limit': 10000},
 {'chunk_size': [1080.0, 1280.0, 687.0],
  'grid_shape': [2, 2, 1],
  'key': 'spatial1',
  'limit': 10000},
 {'chunk_size': [540.0, 640.0, 687.0],
  'grid_shape': [4, 4, 1],
  'key': 'spatial2',
  'limit': 10000},
 {'chunk_size': [270.0, 320.0, 343.5],
  'grid_shape': [8, 8, 2],
  'key': 'spatial3',
  'limit': 10000},
 {'chunk_size': [135.0, 160.0, 171.75],
  'grid_shape': [16, 16, 4],
  'key': 'spatial4',
  'limit': 10000}],
  "upper_bound": [
    2160,
    2560,
    687
  ]
}

In [306]:
info_filename = '/home/ahoag/ngdemo/demo_bucket/test_annotations/test_multispatialindex/info'
with open(info_filename,'w') as outfile:
    json.dump(info,outfile,indent=2)

## Compare to a single spatial index for performance testing

In [270]:
# We already know how to encode just the coordinates. Do it like so for the first 100 points
filename = '/home/ahoag/ngdemo/demo_bucket/test_annotations/an5_singlespatial/spatial0/0_0_0'
# coordinates = converted_points
total_count = len(unique_coordinates)
print(f"Have {total_count} coordinates")
with open(filename,'wb') as outfile:
    buf = struct.pack('<Q',total_count)
    pt_buf = b''.join(struct.pack('<3f',x,y,z) for (x,y,z) in unique_coordinates)
    buf += pt_buf
    id_buf = struct.pack('<%sQ' % len(unique_coordinates), *range(len(unique_coordinates)))
    buf += id_buf
    outfile.write(buf)
print(f"wrote {filename}")

Have 1388798 coordinates
wrote /home/ahoag/ngdemo/demo_bucket/test_annotations/an5_singlespatial/spatial0/0_0_0


In [266]:
coordinates

array([[1440,  948,  219],
       [1421, 1391,  446],
       [1628,  963,  288],
       ...,
       [ 601, 1388,  454],
       [ 461,  751,  431],
       [ 423, 1242,  391]])

In [268]:
# and the info file needs to look like this:
info = {
  "@type": "neuroglancer_annotations_v1",
  "annotation_type": "POINT",
  "by_id": {
    "key": "by_id"
  },
  "dimensions": {
    "x": [
      "5e-06",
      "m"
    ],
    "y": [
      "5e-06",
      "m"
    ],
    "z": [
      "1e-05",
      "m"
    ]
  },
  "lower_bound": [
    0,
    0,
    0
  ],
  "properties": [],
  "relationships": [],
  "spatial": [
    {
      "chunk_size": [
        2160,
        2560,
        687
      ],
      "grid_shape": [
        1,
        1,
        1
      ],
      "key": "spatial0",
      "limit": 10000
    }
  ],
  "upper_bound": [
    2160,
    2560,
    687
  ]
}

In [269]:
info_filename = '/home/ahoag/ngdemo/demo_bucket/test_annotations/an5_singlespatial/info'
with open(info_filename,'w') as outfile:
    json.dump(info,outfile,indent=2)

The number of annotations in the spatially indexed layer is identical to that in the singly indexed layer but it loads way faster! The algorithm is somewhat complicated and hard to debug, but  